In [2]:
import sys
sys.path.insert(1, '../')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_log_error

from agingml import temporal_degradation_test as tdt

/Users/santiago/miniconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def plot_results():
    for model in models:
        errors_df = pd.read_parquet(f'../results/aging/{dataset}/aging_{dataset}_{model}_{n_simulations}_simulations_{n_prod}_prod.parquet')
        d_errors_df = pd.read_parquet(f'../results/aging/{dataset}/aging_{dataset}_{model}_{n_simulations}_simulations_{n_prod}_prod_{freq}.parquet')
        
        errors_df = errors_df[errors_df['model_age'] <= 800]
        d_errors_df = d_errors_df[d_errors_df['model_age'] <= 800]
        test_errors_df = errors_df[errors_df['partition'] == 'test']
        test_mape = test_errors_df.groupby('simulation_id').apply(lambda group: mean_absolute_percentage_error(group.y, group.y_pred))
        test_mape = pd.DataFrame(test_mape, columns=['test_mape']).reset_index()

        d_errors_df = pd.merge(d_errors_df, test_mape, how='left', on='simulation_id')
        d_errors_df = d_errors_df[d_errors_df['test_mape'] <= 0.1]

        valid_models = len(d_errors_df['simulation_id'].drop_duplicates())
        print(f'There are {valid_models} out of {n_simulations}')

        d_errors_df['degradation'] = d_errors_df['error'] > 0.15

        print(d_errors_df['degradation'].value_counts(normalize=True))
        degradations_per_model = d_errors_df.groupby(['simulation_id'])[['degradation']].sum().reset_index()
        num_degradated_models = len(degradations_per_model[degradations_per_model['degradation'] > 5])

        print(f"{num_degradated_models} out of {len(degradations_per_model)} models ({np.round(100 * num_degradated_models / len(degradations_per_model), 1)} %) degradaded")

        print(degradations_per_model.degradation.sort_values())
        fig = tdt.plot_aging_chart(d_errors_df, metric='MAPE', freq='days', plot_name=f"Model Aging Chart: NYC Taxi Trip Demand - {model}")
        fig.show()
        fig.savefig(f'../figures/aging/{dataset}/aging_plot_{dataset}_{model}_{n_simulations}_simulations_{n_prod}_prod.svg', format='svg')

In [4]:
errors_df = pd.read_parquet(f'../results/aging/{dataset}/aging_{dataset}_{models[0]}_{n_simulations}_simulations_{n_prod}_prod.parquet')

In [17]:
errors_df = errors_df[errors_df['model_age'] <= 800]
test_errors_df = errors_df[errors_df['partition'] == 'test']
test_mape = test_errors_df.groupby('simulation_id').apply(lambda group: mean_absolute_percentage_error(group.y, group.y_pred))
test_mape = pd.DataFrame(test_mape, columns=['test_mape']).reset_index()
d_errors_df = test_mape[test_mape['test_mape'] <= 0.1]

valid_model_ids = d_errors_df['simulation_id'].drop_duplicates()

In [21]:
valid_model_ids.values

array([   2,    3,    4, ..., 1497, 1498, 1499])

In [10]:
errors_df

,y,y_pred,partition,timestamp,model_age,is_model_valid,simulation_id
timestamp,,,,,,,
2020-02-27 01:00:00,2329.0,2208.653849,train,2020-02-27 01:00:00,-459,False,0
2020-02-27 02:00:00,1408.0,1324.241803,train,2020-02-27 02:00:00,-459,False,0
2020-02-27 03:00:00,942.0,791.985780,train,2020-02-27 03:00:00,-459,False,0
2020-02-27 04:00:00,1030.0,973.328735,train,2020-02-27 04:00:00,-459,False,0
2020-02-27 05:00:00,1991.0,2237.026396,train,2020-02-27 05:00:00,-459,False,0
...,...,...,...,...,...,...,...
2017-09-18 14:00:00,16228.0,14794.759382,prod,2017-09-18 14:00:00,266,True,1499
2017-09-18 15:00:00,16175.0,14888.364949,prod,2017-09-18 15:00:00,266,True,1499
2017-09-18 16:00:00,15221.0,14910.916928,prod,2017-09-18 16:00:00,266,True,1499


In [12]:
a = errors_df[['simulation_id', 'is_model_valid']].drop_duplicates()
a['is_model_valid'].value_counts()

False    858
True     642
Name: is_model_valid, dtype: int64

In [3]:
dataset = 'taxi'
n_train = 8800 # one year
n_test = 2200
n_prod = 4400
n_simulations = 1500
metric = mean_absolute_percentage_error
freq = 'D'
models =['LGBMRegressor', 'ElasticNet', 'RandomForestRegressor', 'MLPRegressor']
plot_results()